# fMRI analysis with Calcium regressors

modified: july 16

How to run this:
1. First run the *CalciumAnalysis* script on the .lvm file simultaneously acquired with this EPI.
2. Copy the EPI scan in AFNI&ast; format and the *regressor_married.1D* files into the same folder.
3. Within Linux, run this script on a terminal within the scan folder. (Requires AFNI installation, therefore doesn't work on Windows).

All outputs will be in an /analysis subfolder.

&ast; To convert Bruker 2dseq to AFNI BRIK format, run the other python script *convert_bruker_batch_nomove.py* in the original main folder of the animal. It will convert each scan and create BRIK files in the same folders as the 2dseq

TODO: Currently no ROI signal extraction. Create a ROI mask on the template brain to be used with this script in the future

In [1]:
import os
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import subprocess
import shutil
import sys
import numpy as np
import glob

## Settings

Stimulus onset file needs to be present in some folder (or times can be entered below directly). AFNI requirement: All in one line, with onset times in seconds separated by space. NOT considering the initial cutoff.

The *baseline* and *_time* times for the average plots should be same as for previous CalciumAnalysis to allow easily plotting BOLD & Calcium in one figure.

TODO: Some of these parameters should be read from the CalciumAnalysis, instead of typing them again.

In [2]:
stim_duration = 8 #in seconds
TR = 1 #in seconds, e.g. 1.5 for multishot
initial_cutoff = 10 #number of volumes to remove (in absence of dummy scans).

blurFWHM = 0.3 # In [mm]. Rule-of-thumb: 1.5x the voxel resolution.
highpass = 210 # period [s]. Any signal slower than this will be removed. Should be at least 2x inter-stim. interval

# Stimulus File

#stimulus_times = '/media/sf_linuxscripts/stim_files/singleblock.1D' 
#stimulus_times = '/media/sf_linuxscripts/stim_files/suwon_short_8sON_52sOFF.1D'

# alternatively enter the times directly here:
# stimulus_times = "'1D: 180 270 360 450 540 630 720 810 1000 1090 1180 1270 1360 1450 1540 1630'"  #important to have double quotes on both sides like: "'1D: 60 120'"


# For averaging of blocks:

baseline = 10 # baseline to consider before each block
_time = 60 # how many seconds to display after stim block ends

# Location of the EPI template for coregistration (should be a representative animal with same geometry settings)
template_path = '/media/sf_MRIDATA/Template7T/TEMPLATE+orig'

roi_path = '/media/sf_MRIDATA/Template7T/ROIS+orig'


In [5]:
# Alternatively create new stimulus series with starttime (baseline), Number of blocks, and inter-stimulus interval (ISI)
starttime = 120
Nblocks = 15
ISI = 90

stimList=list(range(starttime,starttime+Nblocks*ISI,ISI))
stimString=' '.join(map(str,stimList))
stimulus_times = "'1D: "+stimString+"'"
print(stimulus_times)
# stimulus_times = "'1D: 120 210 300 390 480  '"  #important to have double quotes on both sides like: "'1D: 60 120'"


'1D: 120 210 300 390 480 570 660 750 840 930 1020 1110 1200 1290 1380'


## Selecting scan

In [4]:
if 'file_path' not in vars():
    root = Tk()
    ftypes = [('EPI Scan',"*.BRIK*"),("All Files","*.*")]
    ttl = "Select file"
    dir1 = '/media/sf_MRIDATA'

    root.withdraw()
    file_path = askopenfilename(filetypes = ftypes, initialdir = dir1, title = ttl)
    root.update()
    root.destroy()

    print('loaded '+file_path)

scanname=os.path.splitext(os.path.basename(file_path))[0][:-5]
data_path = os.path.dirname(file_path)  
os.chdir(data_path)


loaded /media/sf_MRIDATA/Test/newAnalysis/E8+orig.BRIK


## Read scan & regressor files
Create a new subfolder /analysis and copy everything in there.

In [5]:
calcium_files = "*regressor*.1D"
# cwd = os.getcwd()
# calcium_regressors = glob.glob(os.path.join(cwd,calcium_files))
calcium_regressors = glob.glob(os.path.join(data_path,calcium_files))

newfolder = os.path.join(data_path,'analyzed')
os.mkdir(newfolder)
for i in calcium_regressors:
    shutil.copy(i,newfolder)

# file1 = scanname+'+orig.BRIK'
# file2= scanname+'+orig.HEAD'
# copy_file1 = os.path.join(file1)
# copy_file2 = os.path.join(file2)
# shutil.copy(copy_file1,newfolder)
# shutil.copy(copy_file2,newfolder)


## AFNI Coregistration & Preprocessing
Currently only coregisters the EPI to another template EPI. But allows to create group maps and consistent/easier ROI drawing.
In the future an atlas could be coregistered to the template.

In [6]:
def runAFNI(inputstring):
    print(inputstring)
    subprocess.run(inputstring,shell=True, check=True)

In [7]:
runAFNI('3dTcat -prefix '+ newfolder + '/' + scanname + ' ' + scanname + "+orig'["+ str(initial_cutoff) + "..$]'")
runAFNI('3dTcat -prefix '+ newfolder + '/single_timepoint' + ' ' + scanname + "+orig'[0]'")

os.chdir(newfolder)

runAFNI("python2.7 /home/virtualfelix/abin/align_epi_anat.py -dset2to1 -dset1 " + template_path + " -dset2 single_timepoint+orig" \
                                " -child_dset2 " + scanname + '+orig ' \
                                " -dset1_strip None -dset2_strip None"  \
                                " -overwrite" \
                                " -big_move" \
                                " -volreg on -volreg_method 3dvolreg" \
                                " -volreg_opts '-Fourier -zpad 1'" \
                                " -Allineate_opts '-maxrot 10 -maxshf 3 -conv 0.005 -twofirst -twoblur 0.8 -source_automask+2 -final wsinc5'" \
                                " -tshift on -tshift_opts '-tzero 0 -quintic'" \
                                " -dset2_base 0" \
                                " -volreg_base 0" \
                                " -suffix .volreg" \
                                " -cost ls")
    
runAFNI('3dTstat -prefix rm.mean' + ' ' + scanname + '.volreg+orig')
runAFNI('3dcalc -a ' + scanname + '.volreg+orig -b rm.mean+orig'\
                    " -expr 'min(200, a/b*100)*step(a)*step(b)' -prefix r" + scanname + '.scale')

3dTcat -prefix /media/sf_MRIDATA/Test/newAnalysis/analysis/E8 E8+orig'[10..$]'
3dTcat -prefix /media/sf_MRIDATA/Test/newAnalysis/analysis/single_timepoint E8+orig'[0]'
python2.7 /home/virtualfelix/abin/align_epi_anat.py -dset2to1 -dset1 /media/sf_MRIDATA/Template7T/TEMPLATE+orig -dset2 single_timepoint+orig -child_dset2 E8+orig  -dset1_strip None -dset2_strip None -overwrite -big_move -volreg on -volreg_method 3dvolreg -volreg_opts '-Fourier -zpad 1' -Allineate_opts '-maxrot 10 -maxshf 3 -conv 0.005 -twofirst -twoblur 0.8 -source_automask+2 -final wsinc5' -tshift on -tshift_opts '-tzero 0 -quintic' -dset2_base 0 -volreg_base 0 -suffix .volreg -cost ls
3dTstat -prefix rm.mean E8.volreg+orig
3dcalc -a E8.volreg+orig -b rm.mean+orig -expr 'min(200, a/b*100)*step(a)*step(b)' -prefix rE8.scale


In [8]:
runAFNI('3dTproject -input r*.scale+orig.BRIK.Z -passband ' + str(1/highpass) + ' 99999 -blur ' + str(blurFWHM) + ' -prefix CleanedData')

3dTproject -input r*.scale+orig.BRIK.Z -passband 0.004761904761904762 99999 -blur 0.3 -prefix CleanedData


## GLM analyses
For now just uses the SPM basis set (SPMG3) as HRF for everything.

1. Just using the **stimulus timing** as regressor (no calcium information).

Output in *statsSPM_el+orig*

In [9]:
runAFNI('3dDeconvolve -input CleanedData+orig' \
                    ' -polort 0' \
                    ' -nodmbase' \
                    ' -overwrite' \
                    ' -num_stimts 1' \
                    ' -stim_times 1 ' +  stimulus_times + \
                    " 'SPMG3(" + str(stim_duration) + ")'" \
                    ' -stim_times_subtract ' + str(initial_cutoff*TR) + \
                    ' -stim_label 1 electro' \
                    ' -iresp 1 HRF_SPM_el' \
                    ' -fout -tout -x1D XSPM.xmat.1D -xjpeg XSPM.jpg' \
                    ' -fitts fittsSPM_el' \
                    ' -errts errtsSPM_el' \
                    ' -bucket statsSPM_el' \
                    ' -cbucket regcoeffsSPM_el')

3dDeconvolve -input CleanedData+orig -polort 0 -nodmbase -overwrite -num_stimts 1 -stim_times 1 '1D: 120 210 300 390 480 570 660 750 840 930 1020 1110 1200 1290 1380 1470 1560 1650 1740' 'SPMG3(8)' -stim_times_subtract 10 -stim_label 1 electro -iresp 1 HRF_SPM_el -fout -tout -x1D XSPM.xmat.1D -xjpeg XSPM.jpg -fitts fittsSPM_el -errts errtsSPM_el -bucket statsSPM_el -cbucket regcoeffsSPM_el


2. Just using the **first calcium channel** as regressor.

Output is *stats_Neuron+orig*

In [10]:
runAFNI('3dDeconvolve -input CleanedData+orig' \
                    ' -polort 0' \
                    ' -nodmbase' \
                    ' -overwrite'	 \
                    ' -num_stimts 1' \
                    ' -stim_times_AM1 1 ' +  calcium_regressors[0] + \
                    " 'SPMG3(" + str(stim_duration) + ")'" \
#                    ' -stim_times_subtract ' + str(initial_cutoff*TR) + \
                    ' -stim_label 1 Neuro' \
                    ' -iresp 1 HRF_SPM_Neuron' \
                    ' -fout -tout' \
                    ' -fitts fitts_Neuron' \
                    ' -bucket stats_Neuron' \
                    ' -cbucket regcoeffs_Neuron')

3dDeconvolve -input CleanedData+orig -polort 0 -nodmbase -overwrite -num_stimts 1 -stim_times_AM1 1 /media/sf_MRIDATA/Test/newAnalysis/regressor1_married.1D 'SPMG3(8)' -stim_label 1 Neuro -iresp 1 HRF_SPM_Neuron -fout -tout -fitts fitts_Neuron -bucket stats_Neuron -cbucket regcoeffs_Neuron


3. Just using the **second calcium channel** as regressor.

Output is *stats_Astro+orig*

In [11]:
runAFNI('3dDeconvolve -input CleanedData+orig' \
                    ' -polort 0' \
                    ' -nodmbase' \
                    ' -overwrite'	 \
                    ' -num_stimts 1' \
                    ' -stim_times_AM1 1 ' + calcium_regressors[1] + \
                    " 'SPMG3(" + str(stim_duration) + ")'" \
#                    ' -stim_times_subtract ' + str(initial_cutoff*TR) + \
                    ' -stim_label 1 Astro' \
                    ' -iresp 1 HRF_SPM_Astro' \
                    ' -fout -tout' \
                    ' -fitts fitts_Astro' \
                    ' -bucket stats_Astro' \
                    ' -cbucket regcoeffs_Astro')

3dDeconvolve -input CleanedData+orig -polort 0 -nodmbase -overwrite -num_stimts 1 -stim_times_AM1 1 /media/sf_MRIDATA/Test/newAnalysis/regressor2_married.1D 'SPMG3(8)' -stim_label 1 Astro -iresp 1 HRF_SPM_Astro -fout -tout -fitts fitts_Astro -bucket stats_Astro -cbucket regcoeffs_Astro


4. Using **both calcium channels** as regressors.

Output is *stats_BothCh+orig*


In [12]:
runAFNI('3dDeconvolve -input CleanedData+orig' \
                    ' -polort 0' \
                    ' -nodmbase' \
                    ' -overwrite' \
                    ' -num_stimts 2' \
                    ' -stim_times_AM1 1 ' + calcium_regressors[0] + \
                    " 'SPMG3(" + str(stim_duration) + ")'" \
                    ' -stim_times_AM1 2 ' + calcium_regressors[1] + \
                    " 'SPMG3(" + str(stim_duration) + ")'" \
#                    ' -stim_times_subtract ' + str(initial_cutoff*TR) + \
                    ' -num_glt 2' \
                    ' -stim_label 1 Neuro' \
                    ' -stim_label 2 Astro' \
                    ' -glt_label 1 NeuroMinusAstro ' \
                    ' -glt_label 2 AstroMinusNEuro ' \
                    " -gltsym 'SYM: +Neuro -Astro'" \
                    " -gltsym 'SYM: +Astro -Neuro'" \
                    ' -fout -tout' \
                    ' -fitts fitts_BothCh' \
                    ' -bucket stats_BothCh')

3dDeconvolve -input CleanedData+orig -polort 0 -nodmbase -overwrite -num_stimts 2 -stim_times_AM1 1 /media/sf_MRIDATA/Test/newAnalysis/regressor1_married.1D 'SPMG3(8)' -stim_times_AM1 2 /media/sf_MRIDATA/Test/newAnalysis/regressor2_married.1D 'SPMG3(8)' -num_glt 2 -stim_label 1 Neuro -stim_label 2 Astro -glt_label 1 NeuroMinusAstro  -glt_label 2 AstroMinusNEuro  -gltsym 'SYM: +Neuro -Astro' -gltsym 'SYM: +Astro -Neuro' -fout -tout -fitts fitts_BothCh -bucket stats_BothCh


4a. Probably unnecessary:  Both calcium channels as regressors but in reverse order.  (To check if order of regressors affects statistics)

Output is *stats_BothCh_reversed+orig*


## Averaging all stimulus blocks

Creates the file:
- average_raw+orig   with the actual BOLD timecourse averaged across all blocks.


In [14]:
k=1
for i in stimList:
    runAFNI('3dTcat -prefix temp_average_raw' + str(k) + ' CleanedData+orig[' + str(int(i) - initial_cutoff - baseline ) + '..' + str(int(i) + stim_duration + _time) +']')
    k += 1

# runAFNI('3dTcat -prefix average_spm' + str(stim_times_all_loaded[0]) + ' fittsSPM_el+orig[' + str(stim_times_all_loaded[0] - baseline ) + '..' + str(stim_times_all_loaded[0] + stim_duration + _time) +']')

runAFNI('3dMean -prefix average_raw temp_average_raw*')

runAFNI("bash -c 'rm -f temp_average* &>/dev/null'")


# Extracting ROIs

In [1]:
runAFNI('3dROIstats -mask ' + roi_path + ' CleanedData+orig > ROIdata.1D')


NameError: name 'runAFNI' is not defined